In [1]:
import tensorflow as tf

In [2]:
import pandas as pd

In [94]:
df=pd.read_csv('spam_ham.csv')
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [95]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


In [96]:
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [16]:
df.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)

In [17]:
df.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [18]:
X=df['v2']
y=df['v1']

In [97]:
X=df['text']
y=df['label_num']

In [104]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [105]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=1000,lower=True)
tokenizer.fit_on_texts(X_train)
X_train=tokenizer.texts_to_sequences(X_train)
X_test=tokenizer.texts_to_sequences(X_test)

In [106]:
X_train[20]

[9,
 52,
 647,
 635,
 1,
 942,
 1,
 16,
 15,
 399,
 14,
 17,
 3,
 824,
 55,
 152,
 311,
 4,
 187,
 398,
 26,
 16,
 1,
 737,
 391,
 12,
 2,
 1,
 418,
 603,
 362,
 85,
 232,
 61,
 105,
 85,
 70,
 3,
 196,
 114,
 452,
 18,
 143,
 44,
 1,
 9,
 647,
 196,
 1,
 17,
 728,
 3,
 152,
 647,
 84,
 26,
 11,
 144,
 111,
 21,
 242,
 8,
 931,
 21,
 242,
 14,
 2,
 7,
 43,
 26,
 55,
 4,
 692,
 2,
 293,
 217,
 684,
 1,
 17,
 66,
 26,
 55,
 285,
 4,
 187,
 242,
 40,
 591,
 16,
 17,
 648,
 3,
 692,
 26,
 380,
 4,
 231,
 254,
 274,
 3,
 239,
 134,
 874,
 97,
 320,
 21,
 16,
 17,
 152,
 728,
 3,
 193,
 376,
 440,
 111,
 21,
 242,
 15,
 399,
 1,
 62,
 6,
 25,
 109,
 1,
 635]

In [108]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_train=pad_sequences(X_train,padding='post',maxlen=100)
X_test=pad_sequences(X_test,padding='post',maxlen=100)
X_train[3:]

array([[  7,   2,  11, ...,   2,  42, 131],
       [  9, 259, 182, ...,   0,   0,   0],
       [  9,   1,  22, ...,   0,   0,   0],
       ...,
       [213,   2, 298, ...,  35, 913, 351],
       [  9,  95,   7, ...,   0,   0,   0],
       [  9,   4, 724, ...,   0,   0,   0]])

In [26]:
from keras.utils import to_categorical
y_train=to_categorical(y_train,num_classes=2)
y_test=to_categorical(y_test,num_classes=2)
y_train

ValueError: invalid literal for int() with base 10: 'ham'

In [27]:
label_mapping = {'ham': 0, 'spam': 1}
y_train_numeric = [label_mapping[label] for label in y_train]
y_test_numeric = [label_mapping[label] for label in y_test]

# Convert numerical labels to one-hot encoded format
y_train_encoded = to_categorical(y_train_numeric, num_classes=2)
y_test_encoded = to_categorical(y_test_numeric, num_classes=2)

print(y_train_encoded)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [1. 0.]
 [1. 0.]
 [1. 0.]]


### Modeling using Sequential model in keras

In [109]:
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,SimpleRNN
from keras import optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

X_train=np.array(X_train).reshape((X_train.shape[0],X_train.shape[1],1))
print(X_train.shape)
X_test=np.array(X_test).reshape((X_test.shape[0],X_test.shape[1],1))
print(X_test.shape)

(4136, 100, 1)
(1035, 100, 1)


In [110]:
def rnn():
    from keras.optimizers import RMSprop
    model=Sequential()
    model.add(SimpleRNN(100,input_shape=(100,1),return_sequences=False))
    model.add(Dense(128,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64,activation='relu'))
    model.add(Dense(32,activation='LeakyReLU'))
    model.add(Dense(2,activation='softmax'))
    model.summary()
    rmsprop=RMSprop(lr=0.01)
    adam=optimizers.Adam(lr=0.001)
    model.compile(loss='categorical_crossentropy',optimizer=rmsprop,metrics=['accuracy'])
    return model
    

In [112]:
from keras.wrappers.scikit_learn import KerasClassifier
model=KerasClassifier(build_fn=rnn,epochs=20,batch_size=100)
model.fit(X_train,y_train)

Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_16 (SimpleRNN)   (None, 100)               10200     
                                                                 
 dense_57 (Dense)            (None, 128)               12928     
                                                                 
 dropout_14 (Dropout)        (None, 128)               0         
                                                                 
 dense_58 (Dense)            (None, 64)                8256      
                                                                 
 dense_59 (Dense)            (None, 32)                2080      
                                                                 
 dense_60 (Dense)            (None, 2)                 66        
                                                                 
Total params: 33,530
Trainable params: 33,530
Non-tra

C:\Users\aksha\AppData\Local\Temp\ipykernel_17624\69373549.py:2: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model=KerasClassifier(build_fn=rnn,epochs=20,batch_size=100)


42/42 [==============================] - 2s 17ms/step - loss: 0.7420 - accuracy: 0.6825
Epoch 2/20
42/42 [==============================] - 1s 15ms/step - loss: 0.6215 - accuracy: 0.6941
Epoch 3/20
42/42 [==============================] - 1s 16ms/step - loss: 0.6085 - accuracy: 0.7041
Epoch 4/20
42/42 [==============================] - 1s 16ms/step - loss: 0.6046 - accuracy: 0.7002
Epoch 5/20
42/42 [==============================] - 1s 17ms/step - loss: 0.5985 - accuracy: 0.7077
Epoch 6/20
42/42 [==============================] - 1s 17ms/step - loss: 0.5995 - accuracy: 0.7041
Epoch 7/20
42/42 [==============================] - 1s 16ms/step - loss: 0.6013 - accuracy: 0.7074
Epoch 8/20
42/42 [==============================] - 1s 16ms/step - loss: 0.5987 - accuracy: 0.7072
Epoch 9/20
42/42 [==============================] - 1s 17ms/step - loss: 0.5956 - accuracy: 0.7079
Epoch 10/20
42/42 [==============================] - 1s 16ms/step - loss: 0.5975 - accuracy: 0.7070
Epoch 11/20
42/42 [=

In [113]:
from sklearn.metrics import accuracy_score
y_pred=model.predict(X_test)
y_pred_encoded=to_categorical(y_pred,num_classes=2)
accuracy_score(y_pred,y_test)

33/33 [==============================] - 0s 4ms/step


0.7159420289855073

### Testing on new Input

In [114]:
a=['r Free! Call The Mobile Update Co FREE on 08002986030']
a=tokenizer.texts_to_sequences(a)
a=np.array(a)
a=pad_sequences(a,padding='post',maxlen=100)
a=a.reshape((a.shape[0],a.shape[1],1))
model.predict(np.array(a))

1/1 [==============================] - 0s 26ms/step


array([0], dtype=int64)

In [115]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[741,   0],
       [294,   0]], dtype=int64)

In [84]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout
from keras.optimizers import RMSprop

def create_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(100, 1), return_sequences=False))
    model.add(Dense(128, activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='softmax'))  # Update the number of units to 2 for binary classification
    model.summary()

    rmsprop = RMSprop(lr=0.001)
    model.compile(loss='categorical_crossentropy', optimizer=rmsprop, metrics=['accuracy'])
    return model




In [86]:
X_train.shape

(4457, 100, 1)

In [87]:
y_train.shape

(4457,)

In [85]:
model = create_model()
model.fit(X_train, y_train)

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_1 (LSTM)               (None, 50)                10400     
                                                                 
 dense_46 (Dense)            (None, 128)               6528      
                                                                 
 dropout_11 (Dropout)        (None, 128)               0         
                                                                 
 dense_47 (Dense)            (None, 64)                8256      
                                                                 
 dense_48 (Dense)            (None, 2)                 130       
                                                                 
Total params: 25,314
Trainable params: 25,314
Non-trainable params: 0
_________________________________________________________________


E:\BROCAMP\DS\dsvenv\lib\site-packages\keras\optimizers\legacy\rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ValueError: in user code:

    File "E:\BROCAMP\DS\dsvenv\lib\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "E:\BROCAMP\DS\dsvenv\lib\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "E:\BROCAMP\DS\dsvenv\lib\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "E:\BROCAMP\DS\dsvenv\lib\site-packages\keras\engine\training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "E:\BROCAMP\DS\dsvenv\lib\site-packages\keras\engine\training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "E:\BROCAMP\DS\dsvenv\lib\site-packages\keras\engine\compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, sample_weight=sw)
    File "E:\BROCAMP\DS\dsvenv\lib